In [1]:
import yfinance as yf

"""
Closing Price
Represents Sj(t)
"""
def closing_price(stockDF, T):
    return stockDF.loc[str(T), 'Close']


"""
Daily Rate of Return.
Represents Yj(t)
"""
def daily_rate_of_return(stockDF, T):
    today = closing_price(stockDF, T)
    yesterday = closing_price(stockDF, T - 1)
    return (today - yesterday) / yesterday


"""
Vj
Creates a list of data corresponding to two weeks of trading days before the day T
"""
def vj(stockDF, T):
    rates_of_return = []
    for i in range(0, 15):
        rates_of_return.append(daily_rate_of_return(stockDF, T - i))
    # Invert the elements in rates_of_return
    rates_of_return = rates_of_return[:: -1]
    return rates_of_return


"""
True Class
returns 0, 1, 2 based on the value of the input
Corresponds to STABLE, UP and DOWN
"""
def true_class(targ):
    # if UP
    if targ >= 0.006:
        return 1
    # if DOWN
    elif targ <= -0.006:
        return 2
    # If STABLE
    else:
        return 0


### TARG is defined as tomorrows Yj(T)
def targ(stockDF, T):
    return daily_rate_of_return(stockDF, T+1)


def make_zt(X):
    return X.map(true_class)

def make_xt(stocks, T):
    """
    Xt Represents an array containing all our line vector stock data "Vj(t)"
    """
    xt_list = []
    ticker_list = []
    for stock, ticker in stocks:
        xt_list.append(vj(stock, T))
        ticker_list.append(ticker)
    return xt_list, ticker_list


In [ ]:
################### Slide 2 #########################
tickers = ["AAPL", "MSFT", "TSLA", "META", "GOOGL", "AMZN", "NVDA", "AMD", "DIS", "NFLX",
           "JPM", "KO", "BAC", "C", "WFC", "GS", "AXP", "MCD", "DJI", "SPY"]

stocks = []

for ticker in tickers:
    stocks.append((yf.Ticker(ticker).history(start='2016-01-01', end='2022-12-31'), ticker))

for stock in stocks:
    new_index = [str(i + 1) for i in range(len(stock[0]))]
    stock[0].index = new_index


In [ ]:
################### Slide 4 #########################
Y = []
for stock, ticker in stocks:
    for day in range(2, len(stock) - 1):
        Y.append((ticker, day, daily_rate_of_return(stock, day)))

In [ ]:
################### Slide 4 cont #########################

import matplotlib.cm as cm

colors = cm.get_cmap('rainbow', 3)  # using 'rainbow' colormap here, you can choose any
unique_tickers = list(set([x[0] for x in Y]))  # gets unique tickers


# Chunking function
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]


# Dividing tickers into chunks of 5
chunks_of_tickers = list(chunks(unique_tickers, 3))
"""
# Now plotting
for chunk in chunks_of_tickers:
    for i, ticker in enumerate(chunk):
        ticker_data = [x for x in Y if x[0] == ticker]  # filter out data for this ticker only
        days = [x[1] for x in ticker_data]  # x-axis data
        rates_of_return = [x[2] for x in ticker_data]  # y-axis data
        plt.plot(days, rates_of_return, color=colors(i), label=ticker)
    plt.xlabel('Day')
    plt.ylabel('Rate of Return')
    plt.title('Rate of Return Over Days for Each Ticker')
    plt.legend()  # add a legend
    plt.show()
"""



In [65]:
################### Slide 5 #########################
Xt, tickers = make_xt(stocks, 97)

# extract vj values and convert to dataframe

import numpy as np
import pandas as pd

# reshape it into 1D and convert into a DataFrame
X_data = pd.DataFrame(Xt, index=tickers).T

print(X_data)

        AAPL      MSFT      TSLA      META     GOOGL      AMZN      NVDA  \
0  -0.001067  0.014839  0.004320  0.008420  0.009225  0.036780  0.013228   
1   0.016446 -0.016400 -0.039206 -0.009615 -0.008357 -0.018323 -0.011944   
2  -0.010402  0.001808 -0.042011  0.005365  0.004136 -0.000626 -0.022772   
3  -0.004059  0.001404 -0.049560 -0.002118  0.004695 -0.017603  0.006042   
4  -0.005577  0.009011  0.016073  0.014260  0.014649  0.022546  0.010295   
5   0.000755 -0.006351 -0.027963 -0.002092  0.005447  0.008606 -0.001415   
6   0.006789  0.018974 -0.001101  0.010567  0.014058  0.034307  0.019274   
7  -0.009741  0.000588  0.001294 -0.008133 -0.011942  0.014451  0.002781   
8  -0.023457  0.009011 -0.008040  0.006359 -0.003395  0.006590 -0.013588   
9   0.001993 -0.008347  0.001592 -0.003908 -0.004450 -0.011157  0.152094   
10  0.037119  0.014683  0.003275 -0.009515  0.007547  0.001042  0.029527   
11 -0.004154 -0.018652 -0.017428 -0.011123 -0.013844 -0.021656  0.002133   
12  0.011445

In [ ]:
################### Slide 6 #########################
Z_data = make_zt(X_data)
print(Z_data)


In [ ]:
################### Slide 7 #########################
# Slide 7 simply describes the goal of the next few slides


In [67]:
################### Slide 8 #########################

CL0 = Z_data.apply(lambda x: (x == 0))
CL1 = Z_data.apply(lambda x: (x == 1))
CL2 = Z_data.apply(lambda x: (x == 2))

s0 = CL0.sum().sum()
s1 = CL1.sum().sum()
s2 = CL2.sum().sum()

N = s0 + s1 + s2

print(N)
print(s0)
print(s1)
print(s2)

R0 = s0/N
R1 = s1/N
R2 = s2/N
print(f"Initial Stable Ratio: {R0}\nInitial Up Ratio: {R1}\nInitial Down Ratio: {R2}")

"""
Due to the data set already containing a fairly balanced set of classes, there is no need to clone the data in X_data
"""


300
112
90
98
Initial Stable Ratio: 0.37333333333333335
Initial Up Ratio: 0.3
Initial Down Ratio: 0.32666666666666666


In [161]:
################### Slide 9 part 1 #########################
# Take all but the last day of trading in X_train
X_train = X_data.iloc[:-1, :]
# Select stock 20 as our target for Random Forest Classifier
target = Z_data.iloc[:,19]
# Take all but the first days rate of return from Y_Train
Y_train = target.iloc[1:]
M = X_train.shape[1]
print(M)

# X_train and Y_train are now dataframes whose elements are offset by one day, producing a training set of a stock's daily value and tomorrow's price action

20


In [162]:
################### Slide 9 part 2 #########################
# 2M/3 proves that M must be a number less than 100, as max_samples must be a number smaller than n_estimators
# I.E., M cannot be a number in the range 300, for a random forest with 100 trees

from sklearn.ensemble import RandomForestClassifier
import time
# Create our random forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42, max_features=18, max_samples=int((2*M)/3), oob_score=True)
start_time = time.time()
clf.fit(X_train, Y_train)
end_time = time.time()

oob_score = clf.oob_score_
elapsed_time = end_time - start_time

print(f"Training took {elapsed_time} seconds.")

print(f"OOB : {oob_score}")
z=clf.predict(X_train)

Training took 0.08609724044799805 seconds.
OOB : 0.21428571428571427


In [129]:
################### Slide 10 part 1 #########################
# Create a reuseable Train Random Forest function to be used for the remaining slides

import time

def TrainRandomForest(num_tree, num_select_features, X_train, Y_train):
    from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
    # Create our random forest classifier
    clf = RandomForestClassifier(n_estimators=num_tree, random_state=42, oob_score=True,
                                 bootstrap=True, max_samples=0.87,
                                 max_features=num_select_features)
    # Calculate computation start time
    start_time = time.time()
    # Train the model
    clf.fit(X_train, Y_train)
    # Calculate computation end time
    end_time = time.time()
    # Total time is end - start
    elapsed_time = end_time - start_time
    # Get out of bag score
    oob_score = clf.oob_score_
    return clf, oob_score, elapsed_time, clf.feature_importances_


In [163]:
################### Slide 10 part 2 #########################
# Compute one iteration of the Random Forest

clf, oob_score, elapsed_time, features = TrainRandomForest(100, 18, X_train, Y_train)
print(f"Training time was {elapsed_time} seconds")

print(f"OOB Score: {oob_score}")

Training time was 0.08740925788879395 seconds
OOB Score: 0.35714285714285715
Features: [0.07164648 0.12719635 0.05135033 0.04709069 0.01924918 0.02779768
 0.01902358 0.0711378  0.01911041 0.03861217 0.03579348 0.13124294
 0.03966387 0.01944193 0.08190082 0.04805506 0.00611002 0.03139941
 0.04609471 0.06808308]


In [112]:
################### Slide 10 part 3 #########################
# Compute the confusion matrix
from sklearn.metrics import confusion_matrix
predicted_z = clf.predict(X_train)
predicted_z = pd.DataFrame(predicted_z)
cmatrix = confusion_matrix(Y_train, predicted_z, labels=[0, 1, 2])
print(cmatrix)


[[3 0 0]
 [0 4 0]
 [0 0 7]]


In [45]:
################### Slide 11 #########################
trees = [100,200,300,400,500]
selected_features = [18,36,72,140,200]
OOB_accuracies = []
computation_times = []
data = []
for tree in trees:
    for SF in selected_features:
        clf, oob_score, elapsed_time = TrainRandomForest(tree, SF, X_train, Y_train)
        data.append((tree, SF, oob_score, elapsed_time))

# print(data)
df = pd.DataFrame(data, columns=['Tree', 'SF', 'oob_score', 'computation_time'])
df_pivot_oob = df.pivot(index='Tree', columns='SF', values='oob_score')
df_pivot_time = df.pivot(index='Tree', columns='SF', values='computation_time')

print(f"OOB\n {df_pivot_oob}")
print(f"Computation times (s)\n {df_pivot_time}")
# Best TR* = 100; Best SF* = 36

OOB
 SF         18        36        72        140       200
Tree                                                  
100   0.428571  0.428571  0.428571  0.428571  0.428571
200   0.428571  0.428571  0.428571  0.428571  0.428571
300   0.500000  0.500000  0.500000  0.500000  0.500000
400   0.500000  0.428571  0.428571  0.428571  0.428571
500   0.500000  0.500000  0.500000  0.500000  0.500000
Computation times (s)
 SF         18        36        72        140       200
Tree                                                  
100   0.117942  0.087714  0.084721  0.084286  0.108509
200   0.212884  0.182407  0.172620  0.173774  0.173813
300   0.254383  0.278475  0.251826  0.252055  0.248673
400   0.329934  0.352914  0.333705  0.333898  0.342823
500   0.422747  0.424908  0.422900  0.421166  0.421869


In [149]:
################### Slide 12-14 #########################
from sklearn.inspection import permutation_importance
RF_prime = RandomForestClassifier(n_estimators=500, max_features=36, criterion='gini', n_jobs=-1, oob_score=True)

RF_prime.fit(X_train, Y_train)

oob_prime = RF_prime.oob_score_
features = RF_prime.n_features_in_
importances = RF_prime.feature_importances_

# Print the feature ranking
print(oob_prime)



0.42857142857142855
Feature ranking:
20
AAPL: 0.012998274530805307
MSFT: 0.025086273983313652
TSLA: 0.08654052748721425
META: 0.2153865990907718
GOOGL: 0.020176004062855395
AMZN: 0.015283337014427804
NVDA: 0.012005774517733278
AMD: 0.24545355785596282
DIS: 0.062063767041473865
NFLX: 0.1048364328712449
JPM: 0.03880931615954919
KO: 0.02499030481243333
BAC: 0.02380482147669236
C: 0.013525901448269616
WFC: 0.009360590052733382
GS: 0.021823673701747977
AXP: 0.02189920875623043
MCD: 0.017433077343974975
DJI: 0.012728539474198445
SPY: 0.01579401831836733


In [ ]:
################### Slide 15 part 1 #########################
print("Feature ranking:")
print(features)

for feature_name, importance in zip(X_train.columns, RF_prime.feature_importances_):
    print(f"{feature_name}: {importance}")


In [150]:
################### Slide 15 part 2 #########################
############# Top 10 most important features ################

# Create tuples of importance, feature
feature_importances = zip(RF_prime.feature_importances_, X_train.columns)

# Sort them by the importance
sorted_feature_importances = sorted(feature_importances, reverse=True)

# Get the ten most important features
top_ten_features = sorted_feature_importances[:10]

# Print the ten most important features
for importance, feature_name in top_ten_features:
    print(f"{feature_name}: {importance}")

AMD: 0.24545355785596282
META: 0.2153865990907718
NFLX: 0.1048364328712449
TSLA: 0.08654052748721425
DIS: 0.062063767041473865
JPM: 0.03880931615954919
MSFT: 0.025086273983313652
KO: 0.02499030481243333
BAC: 0.02380482147669236
AXP: 0.02189920875623043


In [151]:
################### Slide 15 part 3 #########################
############# Top 10 least important features ################
# Create tuples of (importance, feature)
feature_importances = zip(RF_prime.feature_importances_, X_train.columns)

# Sort them by the importance in ascending order
sorted_feature_importances = sorted(feature_importances, key=lambda x: x[0])

# Get the ten least important features
least_important_features = sorted_feature_importances[:10]

# Print the ten least important features
for importance, feature_name in least_important_features:
    print(f"{feature_name}: {importance}")

WFC: 0.009360590052733382
NVDA: 0.012005774517733278
DJI: 0.012728539474198445
AAPL: 0.012998274530805307
C: 0.013525901448269616
AMZN: 0.015283337014427804
SPY: 0.01579401831836733
MCD: 0.017433077343974975
GOOGL: 0.020176004062855395
GS: 0.021823673701747977


In [153]:
################### Slide 16 part 1 ########################
### Create a new dataset of the most important features ####

# Create tuples of importance, feature
feature_importances = zip(RF_prime.feature_importances_, X_train.columns)

# Sort them by the importance
sorted_feature_importances = sorted(feature_importances, reverse=True)

# Get the feature names of ten most important features
top_ten_features = [feature for importance, feature in sorted_feature_importances[:10]]

# Create a new dataset with these features
X_most = X_train[top_ten_features]

In [154]:
################### Slide 16 part 2 ########################
################### Train the new Random Forest model ########################
RF_prime_most = RandomForestClassifier(n_estimators=500, max_features=20, oob_score=True)
RF_prime_most.fit(X_most, Y_train)
print(RF_prime_most.oob_score_)

from sklearn.metrics import confusion_matrix
cmatrix = confusion_matrix(Y_train, predicted_z, labels=[0, 1, 2])
print(cmatrix)


0.5
[[3 0 0]
 [0 4 0]
 [0 0 7]]
